In [138]:
from geopy.distance import distance as geo_distance
from we_recommend_a_grade_10.data import *

test = res_Profile.copy()
test['ratings'] = (Ratings.placeID.drop_duplicates())
test.index = test.placeID
test['num_ratings'] = Ratings.groupby(Ratings.placeID).count()['userID']
display(test.sort_values('num_ratings')[['name', 'num_ratings']])
print("Mean number of ratings is %.2f"%test.num_ratings.mean())

,name,num_ratings
placeID,,
134975,Rincon del Bife,3
132717,tortas hawai,3
135011,Los Vikingos,3
135016,Arrachela Grill,3
132766,Mikasa,3
...,...,...
135052,La Cantina Restaurante,25
132834,Gorditas Doa Gloria,25
135032,Cafeteria y Restaurant El Pacifico,28


Mean number of ratings is 8.93


In [107]:
rating_places = Ratings.placeID.unique()
ratings_unique = rating_places.shape[0]
print("'Ratings' has %s unique places\n"%ratings_unique)

names = "Cuisine;Hours;Parking;Accepts;Profile".split(";")
dfs = [res_Cuisine, res_Hours, res_Parking, res_Accepts, res_Profile]
for i in range(len(dfs)):
    print("'%s' has %s unique places"%(names[i], dfs[i].placeID.unique().shape[0]))
print("\n")
for i in range(len(dfs)):
    dfi = dfs[i].copy()
    dfi.index = dfi.placeID
    num_match = len(dfi.loc[[x for x in rating_places if x in dfi.index]].placeID.unique())
    print("'%s' has %.2f%% coverage on 'Ratings'"%(names[i], 100*num_match / ratings_unique))

grouped = res_Cuisine.groupby("placeID").count()
cuisine_count = grouped.loc[[x for x in rating_places if x in grouped.index]].rename(columns={'Rcuisine':'# cuisines'}).sort_values('# cuisines')

rated_cuisine = res_Cuisine[res_Cuisine.placeID.isin(rating_places)]
cuisine_freq = rated_cuisine.groupby('Rcuisine').count().rename(columns={'placeID':'frequency'}).sort_values('frequency')

print("\nMost restaurants only have 1 cuisine:")
display(cuisine_count)
print("\nThe following cuisines are found in the set of rated restaurants:")
display(cuisine_freq)
print("\nThese are a total of %d tags for %d restaurants"%(cuisine_freq.frequency.sum(), len(cuisine_count)))
print("\nMean cuisine count per restaurant is %.2f"%cuisine_count['# cuisines'].mean())



'Ratings' has 130 unique places

'Cuisine' has 769 unique places
'Hours' has 694 unique places
'Parking' has 675 unique places
'Accepts' has 615 unique places
'Profile' has 130 unique places


'Cuisine' has 73.08% coverage on 'Ratings'
'Hours' has 98.46% coverage on 'Ratings'
'Parking' has 100.00% coverage on 'Ratings'
'Accepts' has 87.69% coverage on 'Ratings'
'Profile' has 100.00% coverage on 'Ratings'

Most restaurants only have 1 cuisine:


,# cuisines
placeID,
135085,1
135059,1
135054,1
135027,1
135039,1
...,...
135041,2
135032,2
135071,2



The following cuisines are found in the set of rated restaurants:


,frequency
Rcuisine,
Vietnamese,1
Armenian,1
Bakery,1
Regional,1
Mediterranean,1
Breakfast-Brunch,1
Game,1
Cafe-Coffee_Shop,1
Contemporary,2



These are a total of 112 tags for 95 restaurants

Mean cuisine count per restaurant is 1.18


### Now creating a distance matrix for ratings

In [170]:
res_coords = res_Profile.copy(); res_coords.index = res_coords.placeID
res_coords = res_coords.drop(columns=[x for x in res_Profile.columns if x not in ['longitude', 'latitude']])
user_coords = user_Profile.copy(); user_coords.index = user_coords.userID
user_coords = user_coords.drop(columns=[x for x in user_Profile.columns if x not in ['longitude', 'latitude']])
display(res_coords.head())
display(user_coords.head())

res_rename = {'latitude': 'res_lat', 'longitude': 'res_lon'}
user_rename = {'latitude': 'user_lat', 'longitude': 'user_lon'}

step1 = Ratings.drop(columns=[x for x in Ratings.columns if "rating" in x])
step2 = pd.merge(step1, res_coords.rename(columns=res_rename), on='placeID',right_index=True).sort_index()
step3 = pd.merge(step2, user_coords.rename(columns=user_rename), on='userID',right_index=True).sort_index()
step3['distance'] = [geo_distance((step3.loc[i].res_lat, step3.loc[i].res_lon),(step3.loc[i].user_lat, step3.loc[i].user_lon)).km for i in step3.index]
matrix = step3[['userID', 'placeID', 'distance']]

def matching(df):
    return (Ratings.userID == df.userID).all()\
           and (Ratings.placeID == df.placeID).all()

print(matching(step1))
print(matching(step2))
print(matching(matrix))

display(matrix)

,latitude,longitude
placeID,,
134999,18.915421,-99.184871
132825,22.147392,-100.983092
135106,22.149709,-100.976093
132667,23.752697,-99.163359
132613,23.752903,-99.165076


,latitude,longitude
userID,,
U1001,22.139997,-100.978803
U1002,22.150087,-100.983325
U1003,22.119847,-100.946527
U1004,18.867000,-99.183000
U1005,22.183477,-100.959891


True
True
True


,userID,placeID,distance
0,U1077,135085,0.693407
1,U1077,135038,0.806876
2,U1077,132825,1.036330
3,U1077,135060,0.729154
4,U1068,135104,0.080878
...,...,...,...
1156,U1043,132630,2.022179
1157,U1011,132715,1.015471
1158,U1068,132733,0.617608
1159,U1068,132594,0.295448
